# Suchoptionen Metadaten

In [112]:
import pandas as pd

"""Einlesen der Metadaten-Tabelle als Dataframe"""
df_metadata = pd.read_csv("files\metadata_letters.csv")

def list_content(column_name):
    """Auflisten der unique-Werte einer Spalte des Dataframe in alphabetischer Reihenfolge"""
    names = (df_metadata[column_name].unique())
    for i in sorted(names):
        print(i)

def print_result(column_name, name):
    """Ausgabe der Zeilen des Dataframe, die ein Suchwort (= name) enthalten"""
    result = df_metadata.loc[df_metadata[column_name] == name, "Dateiname"].to_list()
    for i in result:
        print(i)

## Suche nach Absender*innen

### Ausgeben einer Liste von Absender*innen

In [113]:
list_content("Absender*in")

Aapeli Saarisalo
Benno Landsberger
Cyril John Gadd
François Thureau-Dangin
Friedrich Stummer
Giorgio Castellino
Harri Holma
Jakob J. Finkelstein
Joh. F. Kunstmann
Johannes Pedersen
Karl Budde
Karl Friedrich Müller
Reginald Campbell Thompson
Richard Hartmann
Walter Andrae
Walter G. Kunstmann


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [114]:
#Einfügen des gesuchten Namens
name = "Walter Andrae"

print_result("Absender*in", name)

1932-06-30_Andrae_Landsberger


## Suche nach Empfänger*innen

### Ausgeben einer Liste von Empfänger*innen

In [115]:
list_content("Empfänger*in")

Benno Landsberger
François Thureau-Dangin
Jussi Aro
Karl Friedrich Müller
Torgny Segerstedt


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [116]:
#Einfügen des gesuchten Namens
name = "Jussi Aro"

print_result("Empfänger*in", name)

1960-09-27_Landsberger_Aro


## Suche nach Aufenthaltsorten der Absender*innen

### Ausgeben einer Liste von Aufenthaltsorten

In [117]:
list_content("Aufenthaltsort Absender*in")

Ankara
Berkeley
Berlin
Chicago
Frydek
Heidelberg
Helsinki 
Kopenhagen
Leipzig
London
Marburg
Oxford
Paris
Porto Alegre
Rom
Wiederitzsch 
Würzburg


### Auswahl eines Ortes und Ausgabe der zugehörigen Briefe

In [118]:
#Einfügen des gesuchten Ortes
name = "Leipzig"

print_result("Aufenthaltsort Absender*in", name)

1929-07-10_Mueller_Landsberger


## Suche nach einem Datumszeitraum

In [119]:
#Einfügen eines Datumszeitraums
start_date = "1929-01-01"
end_date = "1931-01-01"

"""Umwandeln des Datums in ein Datetime-Format; Definieren des Datumszeitraums"""
df_metadata["Datum (ISO 8601)"] = pd.to_datetime(df_metadata["Datum (ISO 8601)"], format="ISO8601")
mask = (df_metadata["Datum (ISO 8601)"] > start_date) & (df_metadata["Datum (ISO 8601)"] <= end_date)

result_date = df_metadata.loc[mask, "Dateiname"].to_list()
for i in result_date:
    print(i)

1929-06-06_Landsberger_ThureauDangin
1929-07-10_Mueller_Landsberger
1929-12-09_Hartmann_Landsberger
1930-01-03_Budde_Landsberger


## Suche nach der Briefsprache

### Ausgeben einer Liste von Sprachen

In [120]:
list_content("Sprache")

deutsch
englisch
französisch


### Auswahl einer Sprache und der zugehörigen Briefe

In [121]:
#Einfügen der gesuchten Sprache
name = "englisch"

print_result("Sprache", name)

1933-03-08_Gadd_Landsberger
1943-10-01_Thompson_Landsberger
1956-02-15_Landsberger_Segerstedt
1960-09-27_Landsberger_Aro
1962-02-19_Finkelstein_Landsberger


# Suchoptionen Briefinhalte

In [122]:
import glob
import re
from bs4 import BeautifulSoup
from os.path import join

def content_search(tag, attribute):
    """Einlesen der in TEI kodierten Textinhalte;
    Auffinden von ausgesuchten Tags und Attributen;
    Ausgabe des Ergebnis im Format 'Gesuchtes Tag: Briefname'
    """
    for file in glob.glob(join(f"files", "letters", "tei", "*.xml")):
        with open(file, "r", encoding="utf-8") as infile:
            text = infile.read()
            filename = file.split("\\")[-1].split(".")[0]
            soup = BeautifulSoup(text, "xml")
            tag_list = soup.find_all(tag, attribute)
            tag_tuple_list = []
            for i in tag_list:
                i = i.text
                i = re.sub("-\n", "", i) #Zusammenführung von Wörtern, die am Zeilenende durch Bindestrich getrennt werden
                i = re.sub("\n", " ", i) #Zusammenführung von Bezeichnungen, die aus mehreren Wörtern bestehen und auf zwei oder mehr Zeilen aufgeteilt sind
                i = re.sub("  ", "", i)  #Entfernen von doppelten Leerzeichen
                i = (i, filename)
                tag_tuple_list.append(i)
            tag_tuple_list = list(set(tag_tuple_list))
            tag_tuple_list.sort(key=lambda t : tuple(t[0].lower()))
            for x,y in tag_tuple_list:
                print(x + ": " + y)

### Erwähnungen von modernen Personen

In [123]:
content_search("persName", {"type" : "modern"})

Dr. Schott: 1929-12-09_Hartmann_Landsberger
G.: 1929-12-09_Hartmann_Landsberger
Götze: 1929-12-09_Hartmann_Landsberger
Gadd: 1931-03-11_ThureauDangin_Landsberger
Langdon: 1931-03-11_ThureauDangin_Landsberger
Professeur Zimmern: 1931-03-11_ThureauDangin_Landsberger
Ronzevalle: 1932-03-22_Pedersen_Landsberger
Zimmern: 1932-03-22_Pedersen_Landsberger
Zimmerns: 1932-03-22_Pedersen_Landsberger
Ehelolf: 1932-06-30_Andrae_Landsberger


### Erwähnungen von altorientalischen Personen

In [124]:
content_search("persName", {"type" : "ANE"})


### Erwähnungen von modernen Orten

In [125]:
content_search("placeName", {"type" : "modern"})

Leipzig: 1929-12-09_Hartmann_Landsberger
Marburg: 1929-12-09_Hartmann_Landsberger
Leipzig: 1931-03-11_ThureauDangin_Landsberger
Deutschland: 1932-03-22_Pedersen_Landsberger
Hama: 1932-03-22_Pedersen_Landsberger
Israel: 1932-03-22_Pedersen_Landsberger
Leiden: 1932-03-22_Pedersen_Landsberger
Leipzig: 1932-03-22_Pedersen_Landsberger
London: 1932-03-22_Pedersen_Landsberger


### Erwähnungen von modernen Institutionen

In [126]:
content_search("orgName", {"type" : "academy"})

Sächs. Akademie: 1932-06-30_Andrae_Landsberger


In [127]:
content_search("orgName", {"type" : "institute"})

Orientalischen und am Sprachwissenschaftlichen (indogermanistischen) Seminar: 1929-12-09_Hartmann_Landsberger


### Erwähnungen von altorientalischen Orten

In [128]:
content_search("placeName", {"type" : "ANE"})

Assur: 1929-07-10_Mueller_Landsberger
assyrischen Reiches: 1929-07-10_Mueller_Landsberger
Ras Shamra: 1931-03-11_ThureauDangin_Landsberger


### Erwähnungen von altorientalischen Göttern

In [129]:
content_search("name", {"type" : "deity"})

Assur: 1929-07-10_Mueller_Landsberger
Assurs: 1929-07-10_Mueller_Landsberger
Enlil: 1929-07-10_Mueller_Landsberger
Marduk: 1929-07-10_Mueller_Landsberger
Māratnāri: 1929-07-10_Mueller_Landsberger
Ninlil: 1929-07-10_Mueller_Landsberger
Serua: 1929-07-10_Mueller_Landsberger


### Erwähnungen von Forschungspublikationen

In [130]:
content_search("title", {"type" : "journal"})

MDOG 60: 1929-07-10_Mueller_Landsberger
RA XIV pp. 1 et suiv: 1931-03-11_ThureauDangin_Landsberger
MFOB: 1932-03-22_Pedersen_Landsberger


In [131]:
content_search("title", {"type" : "monograph"})

1er volume de la Neue Folge des Leipziger Semitische Studien: 1931-03-11_ThureauDangin_Landsberger
ana ittišu: 1931-03-11_ThureauDangin_Landsberger
Assur-Zoologie: 1932-06-30_Andrae_Landsberger


### Erwähnungen von Keilschrifttexten

In [132]:
content_search("title", {"type" : "cuneiformtext"})

8. Feldzug Sargons: 1929-07-10_Mueller_Landsberger
KAR 139: 1929-07-10_Mueller_Landsberger
Krönungsritual: 1929-07-10_Mueller_Landsberger
RM. II, 36: 1931-03-11_ThureauDangin_Landsberger


### Erwähnungen von altorientalischen Textpassagen oder Lemmata

### Akkadisch

In [133]:
content_search("foreign", {"xml:lang" : "akk"})

ab-ba-sa: 1929-07-10_Mueller_Landsberger
abnu: 1929-07-10_Mueller_Landsberger
ip-pal: 1929-07-10_Mueller_Landsberger
kararu: 1929-07-10_Mueller_Landsberger
Pû-lišānu: 1929-07-10_Mueller_Landsberger


### Erwähnungen von altorientalischen Sprachen

In [134]:
content_search("lang", "")

Aramäischen: 1929-07-10_Mueller_Landsberger
Akkadisch: 1929-12-09_Hartmann_Landsberger
accadienne: 1931-03-11_ThureauDangin_Landsberger
sumérienne: 1931-03-11_ThureauDangin_Landsberger
aramäische: 1932-03-22_Pedersen_Landsberger
babylonischen: 1932-03-22_Pedersen_Landsberger
sumerischen: 1932-03-22_Pedersen_Landsberger
